In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
import seaborn as sns

%matplotlib inline
#%matplotlib notebook
%reload_ext autoreload
%autoreload 2

from data_utils import *

plt.rcParams['figure.figsize'] = 7, 3

# Range-only datasets for localization 

### Download .mat files and save them in folder ./datasets/

WiFi: http://www.robesafe.es/repository/UAHWiFiDataset/

Landmower: https://github.com/gtrll/gpslam/tree/master/matlab/data

See README for file description. 

## Choose dataset

In [ ]:
anchor_names = None  # use all anchors by default.

#filename = 'datasets/uah1.mat'
# fingers. works ok.
#filename = 'datasets/Plaza1.mat'; # zig zag. does not work super well.
filename = 'datasets/Plaza2.mat' # triangle. works well.

verbose = False
period_it = 10
traj = get_trajectory(filename)
dataname = filename.split('/')[-1].split('.')[0]

if dataname == 'uah1':
    t_window = 1.0
    eps = 2.0
    xlim = 0, 50
    ylim = -20, 20

    min_time = 0
    max_time = 1000

    # for iterative.
    n_complexity_it = 2
    model_it = 'polynomial'
    t_window_it = 80

elif dataname == 'Plaza1':
    t_window = 0.5
    eps = 0.5
    xlim = -50, 10
    ylim = -20, 75

    # choose one:
    min_time = 0  # first big circle
    max_time = 200  # first big circle
    min_time = 510  # first loop
    max_time = 600  # first loop
    min_time = 0  # first few loops
    max_time = 1000  # first few loops.

    # for iterative.
    n_complexity_it = 3
    model_it = 'full_bandlimited'
    period_it = 40
    t_window_it = 20

elif dataname == 'Plaza2':
    t_window = 0.1
    eps = 0.2
    xlim = -80, 10
    ylim = -20, 75

    min_time = 45.1
    period = 100.3 - 45.1
    num_loops = 1
    max_time = min_time + num_loops * period
    traj.period = period

    # for iterative.
    n_complexity_it = 3
    model_it = 'bandlimited'
    period_it = 40
    t_window_it = 40

result_dict = loadmat(filename)

## Prepare dataset

In [ ]:
anchor_data = result_dict['TL']
range_data = result_dict['TD']
gt_data = result_dict['GT']

anchors_df = create_anchors_df(anchor_data)
anchors_df = format_anchors_df(anchors_df, range_system_id=range_system_id, gt_system_id=gt_system_id)

full_df = create_full_df(range_data, gt_data)
full_df = format_data_df(full_df, anchors_df, gt_system_id=gt_system_id, range_system_id=range_system_id)
if verbose:
    print('time going from {:.1f} to {:.1f}'.format(full_df.timestamp.min(), full_df.timestamp.max()))
full_df = full_df[(full_df.timestamp >= min_time) & (full_df.timestamp <= max_time)]
full_df.loc[:, 'timestamp'] = full_df.timestamp - full_df.timestamp.min()

fig, axs = plt.subplots(1, 2)
sns.scatterplot(data=full_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=full_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, ax=axs[1])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')

print('adding ground truth...')
full_df = add_gt_raw(full_df, t_window=t_window, gt_system_id=gt_system_id)
full_df.loc[:, "distance_gt"] = full_df.apply(
    lambda row: apply_distance_gt(row, anchors_df, gt_system_id=gt_system_id), axis=1)
print('...done')

## Plot distance measurements

In [ ]:
fig, axs = plt.subplots(1, 2)
range_df = full_df.loc[full_df.system_id == range_system_id]
sns.scatterplot(data=range_df, x='px', y='py', hue='timestamp', linewidth=0.0, ax=axs[0])
sns.scatterplot(data=anchors_df, x='px', y='py', linewidth=0.0,  ax=axs[0], color='red')
sns.scatterplot(data=range_df, x='timestamp', y='px', hue='timestamp', linewidth=0.0, ax=axs[1])

plot_distance_times(full_df)
ax = plot_distance_errors(full_df)
ax.set_title(dataname)

In [ ]:
plt.figure()
range_df.loc[:, 'distance_error'] = range_df.distance.values - range_df.distance_gt.values
sns.scatterplot(data=range_df, x='px', y='py', hue='distance_error', linewidth=0.0)

In [ ]:
plt.figure()
sns.scatterplot(data=range_df, x='px', y='py', hue='anchor_name', linewidth=0.0)

## Filter measurements

In [ ]:
filtered_df = full_df[full_df.distance<=30]

plot_distance_times(filtered_df)
ax = plot_distance_errors(filtered_df)

## Construct parameters.

In [ ]:
from evaluate_dataset import compute_distance_matrix

chosen_df = full_df
#chosen_df = filtered_df
chosen_distance = 'distance'
#chosen_distance = 'distance_gt'

## Construct anchors. 
if anchor_names is None:
    anchors = anchors_df.loc[:, ['px', 'py', 'pz']].values.astype(np.float32).T
else:
    anchors_df = anchors_df.loc[anchors_df.anchor_name.isin(anchor_names)]
    anchors = get_coordinates(anchors_df, anchor_names)

## Construct times.
range_df = chosen_df[chosen_df.system_id == range_system_id]
times = range_df.timestamp.unique()

## Construct D.
D, times = compute_distance_matrix(chosen_df, anchors_df, anchor_names, times, chosen_distance)
if np.sum(D > 0) > D.shape[0]:
    print('Warning: multiple measurements for times:{}/{}!'.format(
          np.sum(np.sum(D > 0, axis=1)>1), D.shape[0]))

## Construct ground truth.
ground_truth_pos = get_ground_truth(chosen_df, times)

# Global algorithm

In [ ]:
list_complexities = [3, 5, 21, 51]
for n_complexity in list_complexities:

    traj.set_n_complexity(n_complexity)
    basis = traj.get_basis(times=times)

    Chat_weighted = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=True)
    Chat = alternativePseudoInverse(D, anchors[:2, :], basis, weighted=False)

    traj.set_coeffs(coeffs=Chat)

    traj_weighted = traj.copy()
    traj_weighted.set_coeffs(coeffs=Chat_weighted)

    fig, ax = plt.subplots()
    traj.plot(times=times, color='green', label='non-weighted', ax=ax)
    traj_weighted.plot(times=times, color='blue', label='weighted', ax=ax)
    ax.plot(full_df.px, full_df.py, color='black', label='ground truth')
    ax.set_xlabel('x [m]')
    ax.set_ylabel('y [m]')
    ax.set_title('K={}'.format(traj.n_complexity))
    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    ax.legend()

# Iterative algorithms

In [ ]:
traj_it = traj.copy()
traj_it.set_n_complexity(n_complexity_it)
traj_it.model = model_it
traj_it.period = period_it
basis = traj_it.get_basis(times=times)
print('Using trajectory model: \n model={}, K={}, period={}'.format(traj_it.model, traj_it.n_complexity, traj_it.period))

In [ ]:
### Averaging algorithm
print('averaging with time window', t_window_it)
C_list, t_list = averaging_algorithm(D, anchors[:2, :], basis, times, t_window=t_window_it)
ax1 = plot_individual(C_list, t_list, traj_it)
ax1.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]

In [ ]:
### Build up algorithm
C_list, t_list = build_up_algorithm(D, anchors[:2, :], basis, times, eps=eps, verbose=False)
ax1 = plot_individual(C_list, t_list, traj_it.copy())
ax1.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')

result_df = get_smooth_points(C_list, t_list, traj_it)
ax2 = plot_smooth(result_df)
ax2.plot(ground_truth_pos.px, ground_truth_pos.py, color='black')
[[ax.set_xlim(*xlim), ax.set_ylim(*ylim)] for ax in [ax1, ax2]]